### Model Navigator installation

In [1]:
!pip install /workspace/repositories/model_navigator[pyt]

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Processing /workspace/repositories/model_navigator
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for model-navigator: filename=model_navigator-0.2.7-py3-none-any.whl size=464991 sha256=e9a0ea9d1f4abcae66958c48254ddf32917b9c280a522de7ba78a35571e5b6d5
  Stored in directory: /tmp/pip-ephem-wheel-cache-c3ch93mi/wheels/73/49/06/ce9a60907abd2ee285ba78be9c4082887856186224899f6af5
Successfully built model-navigator
  Attempting uninstall: 

### Imports

In [2]:
import subprocess
import sys
import tempfile

import model_navigator as nav
import numpy as np
import torch

/opt/conda/lib/python3.8/site-packages/numpy/core/getlimits.py:499: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/opt/conda/lib/python3.8/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/opt/conda/lib/python3.8/site-packages/numpy/core/getlimits.py:499: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/opt/conda/lib/python3.8/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  return self._float_to_str(self.smallest_subnormal)


### Helper functions

In [3]:
def setup_env(workdir):
    cmd = ["git", "clone", "https://github.com/NVIDIA/DeepLearningExamples", f"{workdir}/DeepLearningExamples"]
    subprocess.run(cmd, check=True)
    sys.path.append(f"{workdir}/DeepLearningExamples/PyTorch/Classification/ConvNets/")

In [4]:
def get_top1_accuracy(runner, dataloader):
    correct, total = 0, 0
    with runner:
        for x, y in iter(dataloader):
            x, y = x.cpu().numpy(), y.cpu().numpy()
            output = runner.infer({"image": x})
            y_pred = np.argmax(output["logits"], axis=1)
            total += len(y)
            correct += np.sum(y_pred == y)
    return correct / total

### Export configuration

In [5]:
MODEL_NAME = "resnet50"
DATA_PATH = "/workspace/repositories/model_navigator/data/"
WORKDIR = "/workspace/repositories/model_navigator/navigator_workdir/"
DEVICE = "cuda"
IMAGE_SIZE = 224
BATCH_SIZE = 16
NUM_CLASSES = 1000
MIN_ACCURACY = 0.785

### Prepare image_classification module

In [6]:
setup_env(WORKDIR)

Cloning into '/workspace/repositories/model_navigator/navigator_workdir//DeepLearningExamples'...


In [7]:
from image_classification import models as convnet_models  # pytype: disable=import-error
from image_classification.dataloaders import get_pytorch_val_loader

In [8]:
MODELS = {
    "resnet50": convnet_models.resnet50,
    "resnext101-32x4d": convnet_models.resnext101_32x4d,
    "se-resnext101-32x4d": convnet_models.se_resnext101_32x4d,
    "efficientnet-b0": convnet_models.efficientnet_b0,
    "efficientnet-b4": convnet_models.efficientnet_b4,
    "efficientnet-widese-b0": convnet_models.efficientnet_widese_b0,
    "efficientnet-widese-b4": convnet_models.efficientnet_widese_b4,
}

### Prepare model and dataloader

In [9]:
model = MODELS[MODEL_NAME](pretrained=True).eval()

val_dataloader, _ = get_pytorch_val_loader(
    DATA_PATH, IMAGE_SIZE, BATCH_SIZE, NUM_CLASSES, False
)


/opt/conda/lib/python3.8/site-packages/torchvision/transforms/transforms.py:321: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  warnings.warn(


### Export
Here we only export to ONNX on CUDA runtime

In [10]:
pkg_desc = nav.torch.export(
    model=model,
    model_name=f"{MODEL_NAME}_pyt",
    dataloader=val_dataloader,
    override_workdir=True,
    target_device=DEVICE,
    input_names=("image",),
    output_names=("logits",),
    runtimes=nav.RuntimeProvider.CUDA,
    target_formats="onnx",
)

/opt/conda/lib/python3.8/site-packages/torch/cuda/__init__.py:121: UserWarning: 
    Found GPU1 NVIDIA GeForce GT 710 which is of cuda capability 3.5.
    PyTorch no longer supports this GPU because it is too old.
    The minimum cuda capability supported by this library is 5.2.
    
  warnings.warn(old_gpu_warn % (d, name, major, minor, min_arch // 10, min_arch % 10))
/opt/conda/lib/python3.8/site-packages/torch/cuda/__init__.py:144: UserWarning: 
NVIDIA GeForce GT 710 with CUDA capability sm_35 is not compatible with the current PyTorch installation.
The current PyTorch install supports CUDA capabilities sm_52 sm_60 sm_61 sm_70 sm_75 sm_80 sm_86 compute_86.
If you want to use the NVIDIA GeForce GT 710 GPU with PyTorch, please check the instructions at https://pytorch.org/get-started/locally/

  warnings.warn(incompatible_device_warn.format(device_name, capability, " ".join(arch_list), device_name))
2022-05-04 11:20:04 INFO     Navigator API: ============================== Config para

### Verification
Verify if each (format, runtime) passes the accuracy test.
PackageDescriptor.get_runner returns [Polygraphy runner](https://github.com/NVIDIA/TensorRT/blob/master/tools/Polygraphy/polygraphy/backend/base/runner.py)

In [11]:
for model_status in pkg_desc.navigator_status.model_status:
    for runtime_results in model_status.runtime_results:
        if runtime_results.status == nav.Status.OK:
            runner = pkg_desc.get_runner(
                format=model_status.format,
                jit_type=model_status.torch_jit,
                precision=model_status.precision,
                runtime=runtime_results.runtime,
            )
            acc = get_top1_accuracy(runner, val_dataloader)
            print(f"For {model_status.format=}, {model_status.torch_jit=}, {model_status.precision=}, {runtime_results.runtime=} accuracy is {acc}.")
            if acc > MIN_ACCURACY:
                pkg_desc.set_verified(
                    format=model_status.format,
                    jit_type=model_status.torch_jit,
                    precision=model_status.precision,
                    runtime=runtime_results.runtime,
                )
                print("Verified.")
            else:
                print(f"Not verified. Min accuracy to be verified is {MIN_ACCURACY}")

2022-05-04 11:21:50 INFO     Navigator API: Loading runner from path: /workspace/repositories/model_navigator/examples/framework/notebooks/navigator_workdir/resnet50_pyt.nav.workspace/onnx/model.onnx


For model_status.format=<Format.ONNX: 'onnx'>, model_status.torch_jit=None, model_status.precision=None, runtime_results.runtime=<RuntimeProvider.CUDA: 'CUDAExecutionProvider'> accuracy is 0.7852.
Verified.


### Save

In [12]:
pkg_desc.save(f"{MODEL_NAME}_pyt.nav", keep_workdir=False, override=True)